# AML - Step 2 & Step 3 Runner (Colab)

This notebook runs Step 2 (feature sanity check) and Step 3 (evaluation) using `scripts/run.py`.

## 1. Setup Environment

In [ ]:
# Clone repository (or upload your code)
!git clone --recursive https://github.com/sapeirone/aml-2025-mistake-detection.git code || echo "Repo already exists or using uploaded code"

# Change to code directory
import os
os.chdir('code')
print(f"Current directory: {os.getcwd()}")

In [ ]:
# Install dependencies
!pip install -q torcheval
!pip install -q -r requirements-cpu.txt 2>/dev/null || pip install -q -r requirements.txt

## 2. Mount Google Drive (Optional)

If your features and checkpoints are in Google Drive, mount it here.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Load Data & Checkpoints

Specify paths to your unzipped directories on Google Drive.

In [ ]:
# ============================================
# UPDATE THESE PATHS TO YOUR DRIVE LOCATIONS
# ============================================

# Path to unzipped omnivore features directory on Drive
# Should contain .npz files directly or in a subdirectory
OMNIVORE_DRIVE_PATH = "/content/drive/MyDrive/path/to/omnivore"  # UPDATE THIS

# Path to unzipped slowfast features directory on Drive
# Should contain .npz files directly or in a subdirectory
SLOWFAST_DRIVE_PATH = "/content/drive/MyDrive/path/to/slowfast"  # UPDATE THIS

# Path to unzipped error_recognition_best checkpoints directory on Drive
# Should contain MLP/ and Transformer/ subdirectories
CHECKPOINTS_DRIVE_PATH = "/content/drive/MyDrive/path/to/error_recognition_best"  # UPDATE THIS

print("Paths configured:")
print(f"  Omnivore: {OMNIVORE_DRIVE_PATH}")
print(f"  SlowFast: {SLOWFAST_DRIVE_PATH}")
print(f"  Checkpoints: {CHECKPOINTS_DRIVE_PATH}")

In [ ]:
# Create data directory structure
!mkdir -p data/video/omnivore
!mkdir -p data/video/slowfast

# Copy omnivore features from Drive
print("Copying Omnivore features...")
!cp -r "{OMNIVORE_DRIVE_PATH}"/* data/video/omnivore/ 2>/dev/null || \
  (echo "Warning: Could not copy from {OMNIVORE_DRIVE_PATH}" && \
   echo "Trying alternative: copying contents if path points to parent directory..." && \
   find "{OMNIVORE_DRIVE_PATH}" -name "*.npz" -exec cp {} data/video/omnivore/ \; 2>/dev/null || true)

# Copy slowfast features from Drive
print("Copying SlowFast features...")
!cp -r "{SLOWFAST_DRIVE_PATH}"/* data/video/slowfast/ 2>/dev/null || \
  (echo "Warning: Could not copy from {SLOWFAST_DRIVE_PATH}" && \
   echo "Trying alternative: copying contents if path points to parent directory..." && \
   find "{SLOWFAST_DRIVE_PATH}" -name "*.npz" -exec cp {} data/video/slowfast/ \; 2>/dev/null || true)

# Verify features were copied
print("\nVerifying features...")
print(f"Omnivore files: $(ls -1 data/video/omnivore/*.npz 2>/dev/null | wc -l) .npz files")
print(f"SlowFast files: $(ls -1 data/video/slowfast/*.npz 2>/dev/null | wc -l) .npz files")

In [ ]:
# Create checkpoints directory
!mkdir -p checkpoints

# Copy checkpoints from Drive
print("Copying checkpoints...")
!cp -r "{CHECKPOINTS_DRIVE_PATH}" checkpoints/error_recognition_best 2>/dev/null || \
  (echo "Warning: Could not copy from {CHECKPOINTS_DRIVE_PATH}" && \
   echo "Trying alternative: if path already points to error_recognition_best directory..." && \
   !cp -r "{CHECKPOINTS_DRIVE_PATH}"/* checkpoints/error_recognition_best/ 2>/dev/null || true)

# Verify checkpoints were copied
print("\nVerifying checkpoints...")
!ls -la checkpoints/error_recognition_best/ 2>/dev/null || echo "Checkpoints directory not found"
print("\nCheckpoint structure:")
!find checkpoints/error_recognition_best -name "*.pt" -type f | head -5 || echo "No .pt files found"

## 4. Step 2: Feature Sanity Check

In [ ]:
# Run Step 2 with default path (data/)
!python scripts/run.py step2

In [ ]:
# Or specify custom features root
# !python scripts/run.py step2 --features_root /path/to/features

## 5. Step 3: Evaluation Reproduction

Run evaluations for different backbones, variants, and splits. Update checkpoint paths with actual epoch numbers.

In [ ]:
# Omnivore - MLP - Step split
!python scripts/run.py step3 --split step --backbone omnivore --variant MLP \
  --ckpt checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_step_epoch_43.pt \
  --threshold 0.6

In [ ]:
# Omnivore - MLP - Recordings split
!python scripts/run.py step3 --split recordings --backbone omnivore --variant MLP \
  --ckpt checkpoints/error_recognition_best/MLP/omnivore/error_recognition_MLP_omnivore_recordings_epoch_XX.pt \
  --threshold 0.4

In [ ]:
# Omnivore - Transformer - Step split
!python scripts/run.py step3 --split step --backbone omnivore --variant Transformer \
  --ckpt checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_step_epoch_XX.pt \
  --threshold 0.6

In [ ]:
# Omnivore - Transformer - Recordings split
!python scripts/run.py step3 --split recordings --backbone omnivore --variant Transformer \
  --ckpt checkpoints/error_recognition_best/Transformer/omnivore/error_recognition_Transformer_omnivore_recordings_epoch_XX.pt \
  --threshold 0.4

In [ ]:
# SlowFast - MLP - Step split
!python scripts/run.py step3 --split step --backbone slowfast --variant MLP \
  --ckpt checkpoints/error_recognition_best/MLP/slowfast/error_recognition_MLP_slowfast_step_epoch_XX.pt \
  --threshold 0.6

In [ ]:
# SlowFast - MLP - Recordings split
!python scripts/run.py step3 --split recordings --backbone slowfast --variant MLP \
  --ckpt checkpoints/error_recognition_best/MLP/slowfast/error_recognition_MLP_slowfast_recordings_epoch_XX.pt \
  --threshold 0.4

In [ ]:
# SlowFast - Transformer - Step split
!python scripts/run.py step3 --split step --backbone slowfast --variant Transformer \
  --ckpt checkpoints/error_recognition_best/Transformer/slowfast/error_recognition_Transformer_slowfast_step_epoch_XX.pt \
  --threshold 0.6

In [ ]:
# SlowFast - Transformer - Recordings split
!python scripts/run.py step3 --split recordings --backbone slowfast --variant Transformer \
  --ckpt checkpoints/error_recognition_best/Transformer/slowfast/error_recognition_Transformer_slowfast_recordings_epoch_XX.pt \
  --threshold 0.4